In [1]:
import feature_extractor as fe 
from os import listdir
import scipy.io as spio
import numpy as np
import dataset

In [ ]:
data, labels = dataset.get_data(1)

218 number of points
1/218: extracting patient_1_47.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]


/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
2/218: extracting patient_1_53.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
3/218: extracting patient_1_84.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
4/218: extracting patient_1_90.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
5/218: extracting patient_1_105.mat
most important channels:  [ 0 72 73 74 75 76 77

most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
33/218: extracting patient_1_41.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
34/218: extracting patient_1_69.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
35/218: extracting patient_1_6.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
36/218: extracting pati

most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
64/218: extracting patient_1_18.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
65/218: extracting patient_1_24.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
66/218: extracting patient_1_30.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channel

most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
94/218: extracting patient_1_36.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
95/218: extracting patient_1_22.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
96/218: extracting patient_1_148.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
97/218: extracting patient_1_174.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 

most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
125/218: extracting patient_1_153.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
126/218: extracting patient_1_184.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
127/218: extracting patient_1_190.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
128/218: extract

most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
155/218: extracting patient_1_140.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
156/218: extracting patient_1_154.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
157/218: extracting patient_1_168.mat
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
158/218: extracting patient_1_16.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 7

most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
186/218: extracting patient_1_67.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
187/218: extracting patient_1_59.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
188/218: extracting patient_1_71.mat
most important channels:  [ 0 72 73 74 75 76 77 78 79 80]
most important channels:  [ 0 73 74 75 76 77 78 79 80 82]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
most important channels:  [ 0 69 68 67 66 65 64 63 62 61]
189/218: extracting

In [ ]:
train_data, train_labels, val_data, val_labels = dataset.train_val_split(data, labels, .7)

print(train_data.shape)
print(train_labels.shape)
print(val_data.shape)
print(val_labels.shape)


In [ ]:
from sklearn.svm import SVC
clf = SVC(C=970, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=10, gamma=.00000000001, kernel='rbf',
    max_iter=1000, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
clf.fit(train_data, train_labels)
y_pred = clf.predict(val_data)

from sklearn.metrics import accuracy_score
print("Training accuracy: ",accuracy_score(clf.predict(train_data), train_labels))
print("Validation accuracy: ",accuracy_score(y_pred, val_labels))